In [45]:
import re
import os
import csv
import pandas as pd
import math

In [46]:
def cleanTweet(tweet):
    tweetDecoded = tweet.encode('ascii', 'ignore').decode('ascii')
    tweetRemovedLinks = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweetDecoded)
    tweetRemovedLinks = tweetRemovedLinks.replace('\n', ' ')
    return tweetRemovedLinks

In [47]:
def openFile(csv):
    tweets = []
    
    df = pd.read_csv(csv)
    
    for index,row in df.iterrows():
        t = row['tweet']
        tweets.append(t)
    
    return tweets

In [48]:
csv = 'aoc.csv'
rawTweets = openFile(csv)
# print(rawTweets)

KeyError: 'tweet'

In [15]:
cleanTweets = []
for tweet in rawTweets:
    cleanTweets.append(cleanTweet(tweet))
# print(cleanTweets)

In [16]:
fileName = csv[:-4] + ".txt"
file = open(fileName, 'w')
file.writelines(cleanTweets)

In [49]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [50]:
# load ascii text and covert to lowercase
filename = "aoc.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  533542
Total Vocab:  62


In [51]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


Total Patterns:  533442


In [52]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
4168/4168 [==============================] - 972s 233ms/step - loss: 2.9752

Epoch 00001: loss improved from inf to 2.89939, saving model to weights-improvement-01-2.8994.hdf5
Epoch 2/20
4168/4168 [==============================] - 953s 229ms/step - loss: 2.7970

Epoch 00002: loss improved from 2.89939 to 2.77894, saving model to weights-improvement-02-2.7789.hdf5
Epoch 3/20
4168/4168 [==============================] - 982s 235ms/step - loss: 2.7356

Epoch 00003: loss improved from 2.77894 to 2.72168, saving model to weights-improvement-03-2.7217.hdf5
Epoch 4/20
4168/4168 [==============================] - 1077s 258ms/step - loss: 2.6804

Epoch 00004: loss improved from 2.72168 to 2.66267, saving model to weights-improvement-04-2.6627.hdf5
Epoch 5/20
4168/4168 [==============================] - 1041s 250ms/step - loss: 2.6130

Epoch 00005: loss improved from 2.66267 to 2.60051, saving model to weights-improvement-05-2.6005.hdf5
Epoch 6/20
4168/4168 [=========================

In [19]:
# load the network weights
filename = "weights-improvement-20-2.1824.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")


Seed:
" f $/mo just as prices are rising.  that alone is reason for @potus to act on student loans asap - w/ "


In [54]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "aoc.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "weights-improvement-20-2.1824.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Total Characters:  533542
Total Vocab:  62
Total Patterns:  533442
Seed:
" y on a "climate corps" to put yits almost as if we helped shape the platform  people must understand "
 the pooeteoein to tee the pooeteoein to tee the call of the pooeteoein to tee the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the call of the pooeteoein to tee po the